# Реализация графовой нейронной сети для предсказания площади и задержи схем

## Подключаем все необходимые библиотеки

In [4]:
#Библиотеки
import pandas as pd
import networkx as nx
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

## Считываем и исследуем данные

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

### Загрузим csv файл с названиями Файлов и целевыми переменными

In [5]:
data = pd.read_csv('D:\\УЧЁБА\\ПРОЕКТ\\предсказание_схемы\\графы\\graphs_10.csv')
#data1 = pd.read_csv('/графы/graphs_11.csv')
#data2 = pd.read_csv('/графы/graphs_12.csv')

### Посмотрим как выглядит наша таблица

In [6]:
data.tail()

,File,Area,Delay
11215,CCGRCG11215,711.59,861.13
11216,CCGRCG11216,53.78,376.55
11217,CCGRCG11217,3.75,80.42
11218,CCGRCG11218,NaN,NaN
11219,CCGRCG11219,NaN,NaN


### Загрузим графы

In [7]:
# Загрузка графов из файлов graphml в список
graphs_dict = {}
for filename in os.listdir('D:\\УЧЁБА\\ПРОЕКТ\\предсказание_схемы\\графы\\gr_10\\10_graphs'):
    if filename.endswith('.graphml'):
        graph = nx.read_graphml(os.path.join('D:\\УЧЁБА\\ПРОЕКТ\\предсказание_схемы\\графы\\gr_10\\10_graphs', filename))
        graphs_dict[filename.split('.graphml')[0]] = graph

In [8]:
graphs_dict['CCGRCG8']

In [9]:
print("Количество узлов:", graphs_dict['CCGRCG8'].number_of_nodes())
print("Количество рёбер:", graphs_dict['CCGRCG8'].number_of_edges())
print("Все узлы графа:", list(graphs_dict['CCGRCG8'].nodes()))
print("Все рёбра графа:", list(graphs_dict['CCGRCG8'].edges()))

Количество узлов: 12
Количество рёбер: 10
Все узлы графа: ['x0', 'f1', 'f2', 'f3', 'f4', 'f5', 'assign0', 'assign1', 'assign2', 'assign3', 'assign4', ' ']
Все рёбра графа: [('x0', 'assign4'), ('assign0', 'f3'), ('assign1', 'f4'), ('assign2', 'f5'), ('assign3', 'f1'), ('assign4', 'f2'), (' ', 'assign0'), (' ', 'assign1'), (' ', 'assign2'), (' ', 'assign3')]


### Преобразуем графы в матрицы смежности

In [10]:
# Обновление значений в словаре graphs_dict
for key, graph in graphs_dict.items():
    # Преобразуйте граф в матрицу смежности
    adjacency_matrix = nx.adjacency_matrix(graph).todense()
    graphs_dict[key] = adjacency_matrix

C:\Users\User\AppData\Local\Temp\ipykernel_20664\753031970.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency_matrix = nx.adjacency_matrix(graph).todense()


## Приведем графы к единому размеру

In [11]:
# Определение максимального размера матрицы
max_size = max(graph.shape[0] for key, graph in graphs_dict.items())
max_size

227

In [12]:
# Дополнение всех матриц нулями до максимального размера
matrices_padded = []
for key, X in graphs_dict.items():
    num_pad_rows = max_size - X.shape[0]
    if num_pad_rows > 0:
        X_padded = np.pad(X, ((0, num_pad_rows), (0, num_pad_rows)), mode='constant')
    else:
        X_padded = X
    graphs_dict[key] = X_padded

## Разделим данные на обучающую и тестовую выборки

In [13]:
# Создаем список для хранения матричных представлений файлов
X = []
# Создаем список для хранения целевых переменных
y = []

# Проходимся по каждой строке в таблице
for index, row in data.iterrows():
    filename = row['File']
    # Проверяем, есть ли матричное представление файла в graphs_dict
    if filename in graphs_dict:
        # Если да, добавляем матричное представление в X
        X.append(graphs_dict[filename])
        # И добавляем соответствующее значение целевой переменной в y
        y.append(row['Delay'])


# Разделяем данные на тренировочный и тестовый датасеты
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Теперь у вас есть X_train, X_test - матричные представления файлов,
# и y_train, y_test - соответствующие им целевые переменные

In [14]:
# Преобразование данных в тензоры PyTorch
X_train = [torch.FloatTensor(X) for X in X_train]
X_test = [torch.FloatTensor(X) for X in X_test]
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

## Создадим модель графовой нейронной сети

In [15]:
# Определение модели GraphRNN
class GraphRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GraphRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_data, input_lengths):
        # Подготовка входных данных для RNN
        packed_input = nn.utils.rnn.pack_padded_sequence(input_data, input_lengths, batch_first=True, enforce_sorted=False)
        # Прямой проход через RNN
        packed_output, _ = self.rnn(packed_input)
        # Распаковка выходных данных
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        # Преобразование выхода RNN к форме (batch_size, output_size)
        output = self.fc(output[:, -1, :])
        return output

# Создание модели
input_size = 227  # Для каждого узла графа мы используем единичное значение
hidden_size = 64
output_size = 1  # Например, для задачи регрессии
model = GraphRNN(input_size, hidden_size, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Обучим нашу модель

In [19]:
# Обучение модели
epochs = 10
for epoch in range(epochs):
    model.train()  # Перевод модели в режим обучения
    running_loss = 0.0
    for X_batch, y_batch in zip(X_train, y_train):
        optimizer.zero_grad()
        # Исправление размерности входных данных
        output = model(X_batch.unsqueeze(0), [X_batch.size(0)])
        loss = criterion(output.squeeze(), y_batch)  # Убираем лишнюю размерность и вычисляем функцию потерь
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(X_train):.4f}')

Epoch [1/10], Loss: 11379152430668930.0000
Epoch [2/10], Loss: 11379151320322290.0000
Epoch [3/10], Loss: 11379150335100796.0000
Epoch [4/10], Loss: 11379148896328516.0000
Epoch [5/10], Loss: 11379147441929676.0000
Epoch [6/10], Loss: 11379153267389606.0000
Epoch [7/10], Loss: 11379150491430072.0000
Epoch [8/10], Loss: 11379146339273548.0000
Epoch [9/10], Loss: 11379156270142716.0000
Epoch [10/10], Loss: 11379148356776090.0000


## Оценим модель на тестовом наборе

In [20]:
# Оценка модели на тестовом наборе
model.eval()  # Перевод модели в режим оценки
with torch.no_grad():
    predictions = []
    for X_batch in X_test:
        output = model(X_batch.unsqueeze(0), [X_batch.size(0)])  # Расширяем размерность входа на 1 и передаем длину
        predictions.append(output.squeeze().item())

## Оценим полученные результаты

In [22]:
# Оценка результатов
predictions = np.array(predictions)
mse = np.mean((predictions - y_test.numpy())**2)
mae = np.mean(predictions - y_test.numpy())
print(f'Mean Squared Error on Test Set: {mse:.4f}')
print(f'Mean Average Error on Test Set: {mae:.4f}')

Mean Squared Error on Test Set: 8644215998319529.0000
Mean Average Error on Test Set: 8643566.8135


## Используем теперь как функцию потерь MAPE

In [16]:
class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()

    def forward(self, y_true, y_pred):
        epsilon = 1e-10  # Защита от деления на ноль
        return torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# Создание модели
input_size = 227  # Для каждого узла графа мы используем единичное значение
hidden_size = 64
output_size = 1  # Например, для задачи регрессии
model = GraphRNN(input_size, hidden_size, output_size)

# Определение функции потерь MAPE и оптимизатора
loss_fn = MAPELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Обучение

In [18]:
# Обучение модели
epochs = 10
for epoch in range(epochs):
    model.train()  # Перевод модели в режим обучения
    running_loss = 0.0
    for X_batch, y_batch in zip(X_train, y_train):
        optimizer.zero_grad()
        # Исправление размерности входных данных
        output = model(X_batch.unsqueeze(0), [X_batch.size(0)])
        loss = loss_fn(output.squeeze(), y_batch)  # Вычисление функции потерь MAPE
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(X_train):.4f}')


Epoch [1/10], Loss: 14539165.4239
Epoch [2/10], Loss: 11020122.5685
Epoch [3/10], Loss: 7289202.8703
Epoch [4/10], Loss: 5374917.9771
Epoch [5/10], Loss: 4242915.1269
Epoch [6/10], Loss: 3498803.5236
Epoch [7/10], Loss: 2973771.0136
Epoch [8/10], Loss: 2584113.8500
Epoch [9/10], Loss: 2283770.8096
Epoch [10/10], Loss: 2045359.8505


## Оценим модель на тестовом наборе

In [19]:
# Оценка модели на тестовом наборе
model.eval()  # Перевод модели в режим оценки
with torch.no_grad():
    predictions = []
    for X_batch in X_test:
        output = model(X_batch.unsqueeze(0), [X_batch.size(0)])  # Расширяем размерность входа на 1 и передаем длину
        predictions.append(output.squeeze().item())

## Оценим полученные результаты

In [20]:
# Оценка результатов
predictions = np.array(predictions)
mape = np.mean(np.abs((predictions - y_test.numpy()) / y_test.numpy())) * 100
print(f'Mean Absolute Percentage Error on Test Set: {mape:.4f}%')

Mean Absolute Percentage Error on Test Set: 290.0938%
